In [1]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../../src/')
from train import load_data
# from diffusion import DiffusionModel
# from evaluate import get_results
from omegaconf import OmegaConf
# from main import load_data, make_model
import numpy as np
import os
import glob
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
import torch
from model2 import Autoencoder, Preprocessor, Discriminator
import magic
import torch
import pathlib
import copy

import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
from omegaconf import OmegaConf
import os
import glob
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from geodesic import jacobian, velocity, CondCurve, GeodesicBridgeOverfit
from plotly3d.plot import scatter, trajectories
import torch
from torch import nn
import pytorch_lightning as pl
from procrustes import Procrustes
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

wandb.login()
api = wandb.Api()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xingzhis. Use `wandb login --relogin` to force relogin


In [2]:
def _distance_to_geodesic_criterion(predicted_geodesic, true_geodesic):
    # the inputs here are single samples from a geodesic; should be shape num_samples x num_dims
    # for each input point, we want the closest distance to any point on the true geodesic using the euclidean distance, torch.cdist
    D = torch.cdist(predicted_geodesic, true_geodesic)
    min_dists_to_true_geodesic = D.min(dim=1)[0]
    # we take the mean of the squared distances
    return torch.mean(min_dists_to_true_geodesic**2)
# def distance_to_geodesic_criterion(
#     predicted_geodesic:torch.Tensor, # size num_geodesics x num_samples x num_dims
#     true_geodesic:torch.Tensor, # size num_geodesics num_samples x num_dims. But it's okay if the num_samples are different
#     ):
#     """
#     Mean of the squared distances from each predicted point to the closest point on the true geodesic
#     """
#     dists = []
#     for i in range(predicted_geodesic.shape[0]):
#         dists.append(_distance_to_geodesic_criterion(predicted_geodesic[i], true_geodesic[i]))
#     dists = torch.stack(dists)
#     return dists.mean()

def distance_to_geodesic_criterion_len(
    predicted_geodesic:torch.Tensor, # size num_geodesics x num_samples x num_dims
    true_geodesic:torch.Tensor, # size num_geodesics num_samples x num_dims. But it's okay if the num_samples are different
    lengths=1.,
    ):
    """
    Mean of the squared distances from each predicted point to the closest point on the true geodesic
    """
    dists = []
    # for i in range(predicted_geodesic.shape[0]):
    for i in range(len(predicted_geodesic)):
        dists.append(_distance_to_geodesic_criterion(predicted_geodesic[i], true_geodesic[i]))
    dists = torch.stack(dists)
    dists = dists / lengths
    return dists.mean()

def distances_to_geodesic(
    predicted_geodesic:torch.Tensor, # size num_geodesics x num_samples x num_dims
    true_geodesic:torch.Tensor, # size num_geodesics num_samples x num_dims. But it's okay if the num_samples are different
    ):
    """
    Mean of the squared distances from each predicted point to the closest point on the true geodesic
    """
    dists = []
    # for i in range(predicted_geodesic.shape[0]):
    for i in range(len(predicted_geodesic)):
        dists.append(_distance_to_geodesic_criterion(predicted_geodesic[i], true_geodesic[i]))
    dists = torch.stack(dists)
    return dists

In [13]:
entity = "xingzhis"
project = "dmae"
# sweep_id = 'neh2vs3e'
sweep_id = 'dcxgbhjp'
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)

In [14]:
names = df[(df['loss_epoch']!='NaN')][['data_name']].values.flatten()

In [15]:
df[(df['loss_epoch']=='NaN')][['data_name']].values.flatten()

array([], dtype=object)

In [16]:

res_list = []
missing = []
failed = []

for data_name in tqdm(names):
    try:
        entity = "xingzhis"
        project = "dmae"
        sweep_id = 'ys48kno0'
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

        runs_data = []

        # Iterate through each run in the sweep
        for run in sweep.runs:
            # Extract metrics and configs
            metrics = run.summary._json_dict
            configs = run.config
            
            # Combine metrics and configs, and add run ID
            combined_data = {**metrics, **configs, "run_id": run.id}
            
            # Append the combined data to the list
            runs_data.append(combined_data)

        # Create a DataFrame from the runs data
        df = pd.DataFrame(runs_data)

        # run_ids = df[(df['data.name'] == data_name) & (df['cfg/loss/weights/cycle'] == 1.) & (df['cfg/dimensions/latent'] == 3)]['run_id']
        run_ids = df[(df['data.name'] == data_name) & (df['loss.weights.cycle'] == 1.0) & (df['dimensions.latent'] == 3)]['run_id']
        # assert len(run_ids) == 1
        if len(run_ids) != 1:
            print('AE not found:', run_ids, data_name)
        run_id = run_ids.iloc[0]
        run = api.run(f"{entity}/{project}/{run_id}")
        # run = api.run(f"{entity}/{project}/{run_id}")
        folder_path = '../../src/wandb/'
        cfg = OmegaConf.create(run.config)
        folder_list = glob.glob(f"{folder_path}*{run.id}*")
        ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        ckpt_path = ckpt_files[0]
        cfg.data.root = '../' + cfg.data.root
        model = Autoencoder.load_from_checkpoint(ckpt_path)
        data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)



        # sweep_id = 'ywep3ixr'
        # sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        # # Initialize an empty list to store run data
        # runs_data = []

        # # Iterate through each run in the sweep
        # for run in sweep.runs:
        #     # Extract metrics and configs
        #     metrics = run.summary._json_dict
        #     configs = run.config
            
        #     # Combine metrics and configs, and add run ID
        #     combined_data = {**metrics, **configs, "run_id": run.id}
            
        #     # Append the combined data to the list
        #     runs_data.append(combined_data)

        # # Create a DataFrame from the runs data
        # df = pd.DataFrame(runs_data)
        # run_ids = df[(df['data.name'] == data_name)][['run_id']]
        # assert len(run_ids) == 1
        # run_id = run_ids.iloc[0]
        # run = api.run(f"{entity}/{project}/{run_ids.iloc[0].values[0]}")
        # folder_path = '../../src/wandb/'
        # cfg = OmegaConf.create(run.config)
        # folder_list = glob.glob(f"{folder_path}*{run.id}*")
        # ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        # ckpt_path = ckpt_files[0]
        # cfg.data.root = '../' + cfg.data.root
        # discriminator = Discriminator.load_from_checkpoint(ckpt_path)



        entity = "xingzhis"
        project = "dmae"
        sweep_id = 'neh2vs3e'
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        runs_data = []

        # Iterate through each run in the sweep
        for run in sweep.runs:
            # Extract metrics and configs
            metrics = run.summary._json_dict
            configs = run.config
            
            # Combine metrics and configs, and add run ID
            combined_data = {**metrics, **configs, "run_id": run.id}
            
            # Append the combined data to the list
            runs_data.append(combined_data)

        # Create a DataFrame from the runs data
        df = pd.DataFrame(runs_data)

        run_ids = df[(df['data_name'] == data_name) & (df['loss_epoch'] != 'NaN') & (df['dimensions_latent'] == 3)]['run_id']
        # assert len(run_ids) == 1
        if len(run_ids) != 1:
            print('Geod not found', run_ids, data_name)
        run_id = run_ids.iloc[0]
        run = api.run(f"{entity}/{project}/{run_id}")
        cfg_main = OmegaConf.create(run.config)
        folder_path = '../geodesic_on_datasets//wandb/'
        folder_list = glob.glob(f"{folder_path}*{run.id}*")
        ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        ckpt_path = ckpt_files[0]

        x = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
        xbatch = torch.tensor(data['start_points'], dtype=x.dtype, device=x.device)
        xendbatch = torch.tensor(data['end_points'], dtype=x.dtype, device=x.device)
        # xbatch = model.encoder.preprocessor.normalize(xbatch)
        # xendbatch = model.encoder.preprocessor.normalize(xendbatch)
        # if cfg_main.overfit:
        #     ids = torch.eye(xbatch.size(0))
        # else:
        ids = torch.zeros((xbatch.size(0),1))
        dataset = TensorDataset(xbatch, xendbatch, ids)
        dataloader = DataLoader(dataset, batch_size=len(x), shuffle=False)

        for param in model.encoder.parameters():
            param.requires_grad = False
        # def func(x):
        #     return model.encoder(x)
        # for param in discriminator.parameters():
        #     param.requires_grad = False
        # def discriminator_func_for_grad(x):
        #     return discriminator.positive_proba(x, normalize=False).reshape(-1,1)
        # def discriminator_func(x):
        #     return discriminator.positive_proba(x, normalize=False).reshape(-1,1)
        ofm = lambda x: x
        gbmodel = GeodesicBridgeOverfit.load_from_checkpoint(
            checkpoint_path=ckpt_path,
            func=ofm,
            # func = enc_func,
            # discriminator_func=disc_func_pen,
            # discriminator_func_for_grad=discriminator_func_for_grad,
            input_dim=x.size(1), 
            hidden_dim=64, 
            scale_factor=1, 
            symmetric=True, 
            num_layers=3, 
            n_tsteps=100, 
            lr=1e-3, 
            weight_decay=1e-3,
            discriminator_weight=0.,
            discriminator_func_for_grad_weight=0.,
            id_dim=1,
            id_emb_dim=1,
            density_weight=0.,
            length_weight=1.,
        )

        # batch = next(iter(dataloader))
        # x0, x1, ids = batch
        try:
            data_gt = np.load(f"/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/dmae/data/neurips_results/toy/gt/{data_name}.npz", allow_pickle=True)
        except:
            print(f"CANNOT FIND FILE {data_name}.npz")
            missing.append(data_name)
            continue
        xbatch = torch.tensor(data_gt['start_points'], dtype=x.dtype, device=x.device)
        xendbatch = torch.tensor(data_gt['end_points'], dtype=x.dtype, device=x.device)
        x0 = xbatch
        x1 = xendbatch
        # xbatch = model.encoder.preprocessor.normalize(xbatch)
        # xendbatch = model.encoder.preprocessor.normalize(xendbatch)
        ids = torch.zeros((xbatch.size(0),1))
        # ids = torch.eye((xbatch.size(0)))

        # dataset = TensorDataset(xbatch, xendbatch, ids)
        # dataloader = DataLoader(dataset, batch_size=len(z), shuffle=True)

        def cc_func(x0, x1, t):
            return gbmodel.cc(x0, x1, t, ids)
        vectors = velocity(cc_func, gbmodel.ts, x0, x1)
        cc_pts = gbmodel.cc(x0, x1, gbmodel.ts, ids)
        vectors_flat = vectors.flatten(0,1)
        cc_pts_flat = cc_pts.flatten(0, 1)
        jac_flat = jacobian(gbmodel.func, cc_pts_flat)
        length_all = torch.sqrt((torch.einsum("nij,nj->ni", jac_flat, vectors_flat)**2).sum(axis=1))
        length_all = length_all.reshape(vectors.shape[0], vectors.shape[1])
        length = length_all.mean(axis=0)

        geods = (cc_pts_flat).reshape(cc_pts.shape)
        length2 = torch.sqrt(torch.diff(geods, axis=0)**2).sum(axis=-1).sum(axis=0)


        # plt.scatter(length2.detach().numpy(), data_gt['geodesic_lengths'])
        # plt.title(data_name)
        # plt.show()

        # gt_len = torch.tensor(data_gt['geodesic_lengths'])
        true_geod = torch.tensor(data_gt['geodesics'], dtype=torch.float32)
        gt_len = torch.sqrt(torch.diff(true_geod.permute(1,0,2), axis=0)**2).sum(axis=-1).sum(axis=0)
        corr = np.corrcoef(length.detach().numpy(), gt_len.cpu().numpy())[0,1]
        mse = ((length.detach().numpy() - gt_len.cpu().numpy())**2).mean()
        dist2geod = distance_to_geodesic_criterion_len(geods.permute(1,0,2), true_geod, lengths=gt_len).detach().numpy()

        res_list.append(dict(
            name=data_name,
            length_corr=corr,
            length_mse=mse,
            dist2geod=dist2geod
        ))
    except Exception as e:
        failed.append(data_name)
        print(e)

res_df = pd.DataFrame(res_list)


  0%|          | 0/36 [00:00<?, ?it/s]

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.
  rank_zero_warn(
 39%|███▉      | 14/36 [00:08<00:10,  2.15it/s]

Geod not found Series([], Name: run_id, dtype: object) saddle_10_0.1
single positional indexer is out-of-bounds


 42%|████▏     | 15/36 [00:08<00:08,  2.56it/s]

Geod not found 9     z5dzkrwu
22    z5dzkrwu
Name: run_id, dtype: object hemisphere_50_0.1


 56%|█████▌    | 20/36 [00:11<00:05,  2.67it/s]

list index out of range


 72%|███████▏  | 26/36 [00:14<00:03,  2.55it/s]

Geod not found Series([], Name: run_id, dtype: object) hemisphere_15_0.5
single positional indexer is out-of-bounds


100%|██████████| 36/36 [00:19<00:00,  1.88it/s]


In [17]:
failed

['saddle_10_0.1', 'saddle_50_0.1', 'hemisphere_15_0.5']

In [19]:

res_list = []
missing = []
failed2 = []

for data_name in tqdm(failed):
    try:
        entity = "xingzhis"
        project = "dmae"
        sweep_id = 'ys48kno0'
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

        runs_data = []

        # Iterate through each run in the sweep
        for run in sweep.runs:
            # Extract metrics and configs
            metrics = run.summary._json_dict
            configs = run.config
            
            # Combine metrics and configs, and add run ID
            combined_data = {**metrics, **configs, "run_id": run.id}
            
            # Append the combined data to the list
            runs_data.append(combined_data)

        # Create a DataFrame from the runs data
        df = pd.DataFrame(runs_data)

        # run_ids = df[(df['data.name'] == data_name) & (df['cfg/loss/weights/cycle'] == 1.) & (df['cfg/dimensions/latent'] == 3)]['run_id']
        run_ids = df[(df['data.name'] == data_name) & (df['loss.weights.cycle'] == 1.0) & (df['dimensions.latent'] == 3)]['run_id']
        # assert len(run_ids) == 1
        if len(run_ids) != 1:
            print('AE not found:', run_ids, data_name)
        run_id = run_ids.iloc[0]
        run = api.run(f"{entity}/{project}/{run_id}")
        # run = api.run(f"{entity}/{project}/{run_id}")
        folder_path = '../../src/wandb/'
        cfg = OmegaConf.create(run.config)
        folder_list = glob.glob(f"{folder_path}*{run.id}*")
        ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        ckpt_path = ckpt_files[0]
        cfg.data.root = '../' + cfg.data.root
        model = Autoencoder.load_from_checkpoint(ckpt_path)
        data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)



        # sweep_id = 'ywep3ixr'
        # sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        # # Initialize an empty list to store run data
        # runs_data = []

        # # Iterate through each run in the sweep
        # for run in sweep.runs:
        #     # Extract metrics and configs
        #     metrics = run.summary._json_dict
        #     configs = run.config
            
        #     # Combine metrics and configs, and add run ID
        #     combined_data = {**metrics, **configs, "run_id": run.id}
            
        #     # Append the combined data to the list
        #     runs_data.append(combined_data)

        # # Create a DataFrame from the runs data
        # df = pd.DataFrame(runs_data)
        # run_ids = df[(df['data.name'] == data_name)][['run_id']]
        # assert len(run_ids) == 1
        # run_id = run_ids.iloc[0]
        # run = api.run(f"{entity}/{project}/{run_ids.iloc[0].values[0]}")
        # folder_path = '../../src/wandb/'
        # cfg = OmegaConf.create(run.config)
        # folder_list = glob.glob(f"{folder_path}*{run.id}*")
        # ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        # ckpt_path = ckpt_files[0]
        # cfg.data.root = '../' + cfg.data.root
        # discriminator = Discriminator.load_from_checkpoint(ckpt_path)



        entity = "xingzhis"
        project = "dmae"
        sweep_id = '1b2p375i'
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        runs_data = []

        # Iterate through each run in the sweep
        for run in sweep.runs:
            # Extract metrics and configs
            metrics = run.summary._json_dict
            configs = run.config
            
            # Combine metrics and configs, and add run ID
            combined_data = {**metrics, **configs, "run_id": run.id}
            
            # Append the combined data to the list
            runs_data.append(combined_data)

        # Create a DataFrame from the runs data
        df = pd.DataFrame(runs_data)

        run_ids = df[(df['data_name'] == data_name) & (df['loss_epoch'] != 'NaN') & (df['dimensions_latent'] == 3)]['run_id']
        # assert len(run_ids) == 1
        if len(run_ids) != 1:
            print('Geod not found', run_ids, data_name)
        run_id = run_ids.iloc[0]
        run = api.run(f"{entity}/{project}/{run_id}")
        cfg_main = OmegaConf.create(run.config)
        folder_path = '../geodesic_on_datasets//wandb/'
        folder_list = glob.glob(f"{folder_path}*{run.id}*")
        ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        ckpt_path = ckpt_files[0]

        x = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
        xbatch = torch.tensor(data['start_points'], dtype=x.dtype, device=x.device)
        xendbatch = torch.tensor(data['end_points'], dtype=x.dtype, device=x.device)
        # xbatch = model.encoder.preprocessor.normalize(xbatch)
        # xendbatch = model.encoder.preprocessor.normalize(xendbatch)
        # if cfg_main.overfit:
        #     ids = torch.eye(xbatch.size(0))
        # else:
        ids = torch.zeros((xbatch.size(0),1))
        dataset = TensorDataset(xbatch, xendbatch, ids)
        dataloader = DataLoader(dataset, batch_size=len(x), shuffle=False)

        for param in model.encoder.parameters():
            param.requires_grad = False
        # def func(x):
        #     return model.encoder(x)
        # for param in discriminator.parameters():
        #     param.requires_grad = False
        # def discriminator_func_for_grad(x):
        #     return discriminator.positive_proba(x, normalize=False).reshape(-1,1)
        # def discriminator_func(x):
        #     return discriminator.positive_proba(x, normalize=False).reshape(-1,1)
        ofm = lambda x: x
        gbmodel = GeodesicBridgeOverfit.load_from_checkpoint(
            checkpoint_path=ckpt_path,
            func=ofm,
            # func = enc_func,
            # discriminator_func=disc_func_pen,
            # discriminator_func_for_grad=discriminator_func_for_grad,
            input_dim=x.size(1), 
            hidden_dim=64, 
            scale_factor=1, 
            symmetric=True, 
            num_layers=3, 
            n_tsteps=100, 
            lr=1e-3, 
            weight_decay=1e-3,
            discriminator_weight=0.,
            discriminator_func_for_grad_weight=0.,
            id_dim=1,
            id_emb_dim=1,
            density_weight=0.,
            length_weight=1.,
        )

        # batch = next(iter(dataloader))
        # x0, x1, ids = batch
        try:
            data_gt = np.load(f"/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/dmae/data/neurips_results/toy/gt/{data_name}.npz", allow_pickle=True)
        except:
            print(f"CANNOT FIND FILE {data_name}.npz")
            missing.append(data_name)
            continue
        xbatch = torch.tensor(data_gt['start_points'], dtype=x.dtype, device=x.device)
        xendbatch = torch.tensor(data_gt['end_points'], dtype=x.dtype, device=x.device)
        x0 = xbatch
        x1 = xendbatch
        # xbatch = model.encoder.preprocessor.normalize(xbatch)
        # xendbatch = model.encoder.preprocessor.normalize(xendbatch)
        ids = torch.zeros((xbatch.size(0),1))
        # ids = torch.eye((xbatch.size(0)))

        # dataset = TensorDataset(xbatch, xendbatch, ids)
        # dataloader = DataLoader(dataset, batch_size=len(z), shuffle=True)

        def cc_func(x0, x1, t):
            return gbmodel.cc(x0, x1, t, ids)
        vectors = velocity(cc_func, gbmodel.ts, x0, x1)
        cc_pts = gbmodel.cc(x0, x1, gbmodel.ts, ids)
        vectors_flat = vectors.flatten(0,1)
        cc_pts_flat = cc_pts.flatten(0, 1)
        jac_flat = jacobian(gbmodel.func, cc_pts_flat)
        length_all = torch.sqrt((torch.einsum("nij,nj->ni", jac_flat, vectors_flat)**2).sum(axis=1))
        length_all = length_all.reshape(vectors.shape[0], vectors.shape[1])
        length = length_all.mean(axis=0)

        geods = (cc_pts_flat).reshape(cc_pts.shape)
        length2 = torch.sqrt(torch.diff(geods, axis=0)**2).sum(axis=-1).sum(axis=0)


        # plt.scatter(length2.detach().numpy(), data_gt['geodesic_lengths'])
        # plt.title(data_name)
        # plt.show()

        # gt_len = torch.tensor(data_gt['geodesic_lengths'])
        true_geod = torch.tensor(data_gt['geodesics'], dtype=torch.float32)
        gt_len = torch.sqrt(torch.diff(true_geod.permute(1,0,2), axis=0)**2).sum(axis=-1).sum(axis=0)
        corr = np.corrcoef(length.detach().numpy(), gt_len.cpu().numpy())[0,1]
        mse = ((length.detach().numpy() - gt_len.cpu().numpy())**2).mean()
        dist2geod = distance_to_geodesic_criterion_len(geods.permute(1,0,2), true_geod, lengths=gt_len).detach().numpy()

        res_list.append(dict(
            name=data_name,
            length_corr=corr,
            length_mse=mse,
            dist2geod=dist2geod
        ))
    except Exception as e:
        failed2.append(data_name)
        print(e)

res_df2 = pd.DataFrame(res_list)


100%|██████████| 3/3 [00:02<00:00,  1.45it/s]

Geod not found Series([], Name: run_id, dtype: object) hemisphere_15_0.5
single positional indexer is out-of-bounds


In [20]:
results_ours = pd.concat([res_df, res_df2], ignore_index=True)

In [21]:
results_ours.sort_values('name')

,name,length_corr,length_mse,dist2geod
23,ellipsoid_10_0.1,0.468024,6.542765,0.14057913
31,ellipsoid_15_0,0.913999,13.506490,0.024221402
30,ellipsoid_15_0.1,0.845484,19.020750,0.05549545
32,ellipsoid_15_0.3,0.610557,14.293184,0.30313063
22,ellipsoid_15_0.5,0.556689,12.079306,0.56921166
25,ellipsoid_15_0.7,-0.073554,12.685026,1.1486657
29,ellipsoid_50_0.1,0.629106,25.590904,0.13367188
24,ellipsoid_5_0.1,0.501222,9.889479,0.059939913
26,ellipsoid_none_0.1,0.273180,5.330163,0.1547515
18,hemisphere_10_0.1,0.976843,2.880428,0.048764646


In [22]:
results_ours

,name,length_corr,length_mse,dist2geod
0,torus_5_0.1,0.904453,9.136180,0.05429554
1,torus_none_0.1,0.835447,3.580664,0.190402
2,torus_15_0.7,0.522471,15.566595,0.95686424
3,torus_15_0.1,0.982191,28.077686,0.05876852
4,torus_50_0.1,0.925115,33.913094,0.10959925
5,torus_15_0.3,0.903769,22.883854,0.22196908
6,torus_15_0.5,0.866086,18.514904,0.50089777
7,torus_10_0.1,0.940233,15.396060,0.065491825
8,saddle_15_0.1,0.869598,3.940537,0.050379395
9,saddle_15_0.7,0.254303,3.995860,1.7504785


In [23]:
results_ours.to_csv('geodesics_no_density.csv', index=False)

In [25]:
failed2

['hemisphere_15_0.5']

: 